In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from tensorflow import keras 
import tensorflow.keras.preprocessing.text as tfkpt
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
import csv
import json

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
bird_df = pd.read_csv('~/Desktop/training.1600000.processed.noemoticon.csv', encoding='latin-1')
bird_df.columns = ['score', '', 'date', '', 'usr', 'review'] 
bird_training_df = bird_df[['score', 'review']].dropna()
bird_training_df.sample(5)

,score,review
1050821,4,@auburnjones yeaaaah tonight gonna be fun
1476194,4,"Okay, I lied, I guess I can just change my twi..."
1044678,4,@DudeAsInCool Its not that bad. DMV Tip: Sched...
1153001,4,@elizamanson Heeeeey how are you? you going to...
18938,0,"Race started,leader is The Pacecar #F1"


In [32]:
def create_sentiment(int):
    if int == 4 or int == 3: 
        return 2
    if int == 2: 
        return 1
    if int == 1 or int == 0: 
        return 0    


In [33]:
bird_training_df['sentiment'] = bird_training_df['score'].apply(create_sentiment)

bird_training_df.sample(5)

,score,review,sentiment
605735,0,@leevk yeah... gonna do that after the call......,0
1530222,4,&amp;&amp; if you sippin' on Patron with the f...,2
978387,4,@xjujubeanx eating baked mac n cheese bout to...,2
250173,0,@aianna21 I'd rather not disclose that here. B...,0
1188813,4,@Spotter5 oooh what is that,2


In [34]:
bird_y = bird_training_df.sentiment
print(bird_y)

0          0
1          0
2          0
3          0
4          0
          ..
1599994    2
1599995    2
1599996    2
1599997    2
1599998    2
Name: sentiment, Length: 1599999, dtype: int64


In [5]:
bird_x = bird_training_df.review
print(bird_x)

0          is upset that he can't update his Facebook by ...
1          @Kenichan I dived many times for the ball. Man...
2            my whole body feels itchy and like its on fire 
3          @nationwideclass no, it's not behaving at all....
4                              @Kwesidei not the whole crew 
                                 ...                        
1599994    Just woke up. Having no school is the best fee...
1599995    TheWDB.com - Very cool to hear old Walt interv...
1599996    Are you ready for your MoJo Makeover? Ask me f...
1599997    Happy 38th Birthday to my boo of alll time!!! ...
1599998    happy #charitytuesday @theNSPCC @SparksCharity...
Name: review, Length: 1599999, dtype: object


In [6]:
bird_training_df.describe()

,score
count,1.599999e+06
mean,2.000001e+00
std,2.000001e+00
min,0.000000e+00
25%,0.000000e+00
50%,4.000000e+00
75%,4.000000e+00
max,4.000000e+00


In [7]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(bird_x)
tokenizer.get_config()

, "tzeyan": 690036, "fineness": 690037, "goodingoodout": 690038, "agboise": 690039, "flowerparts74": 690040, "classiccccccccccccccccc": 690041, "waymansl": 690042, "sbtdl": 690043, "\'bulance\'": 690044, "williamsondavid": 690045, "taraatbeyond": 690046, "vloggercon": 690047, "ofbe": 690048, "everythanggg": 690049, "donaliza": 690050, "deananddeluca": 690051, "manuelrauber": 690052, "mrsegrams": 690053, "nyq4xs": 690054, "11e6te": 690055, "7iuns": 690056, "0psq4j": 690057, "mitchmussonews": 690058, "7j1ww": 690059, "marybethjbellon": 690060, "coolchic8897": 690061, "yge": 690062, "bomiqbt5nzy": 690063, "prozess": 690064, "pletchum": 690065, "devanne": 690066, "jatingupta25": 690067, "pikle": 690068, "charlottechc": 690069, "swooooosh": 690070, "nayala": 690071, "hereiblog": 690072, "ireyyyy": 690073, "commonjason": 690074, "praie": 690075, "eastward": 690076, "bessey": 690077, "7jlh9": 690078, "seleeenaa": 690079, "demi\'ll": 690080, "miley\'ll": 690081, "psycht": 690082, "7jlkc": 6900

In [8]:
bird_x = tokenizer.texts_to_sequences(bird_x)


In [9]:
bird_x = tokenizer.sequences_to_matrix(bird_x)


In [36]:
bird_y = keras.utils.to_categorical(bird_y, 3)

In [26]:
model = Sequential()
#Layer 1: activation - Sigmoid, dropout 1 at 10%
model.add(Dense(677, activation='sigmoid'))
model.add(Dropout(0.1))
#Layer 2: activation - Rectified Linear Unit, dropout 1 at 20%
model.add(Dense(557, activation='relu'))
model.add(Dropout(0.2))
#Layer 3: activation - Linear, dropout 1 at 30%
model.add(Dense(419, 'linear'))
model.add(Dropout(0.3))
#Layer 4: activation - Exponential Linear Unit, dropout 1 at 40%
model.add(Dense(281, activation='elu'))
model.add(Dropout(0.4))
#Layer 5: activation - Softmax, no dropout
model.add(Dense(3, activation='softmax'))

In [28]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
model.fit(bird_x, bird_y, 
batch_size=37, 
epochs=7, 
verbose=1, 
validation_split=0.2)

Train on 1279999 samples, validate on 320000 samples
Epoch 1/7
    629/1279999 [..............................] - ETA: 189:19:19 - loss: 0.9997 - accuracy: 0.5236

KeyboardInterrupt: 

In [ ]:
bird_model = model.to_json()
with open('bird_model.json', 'w') as json_file: 
  json_file.write(model_json)
model.save_weights('bird_model.h5') 